In [47]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [48]:
import pyspiel
import open_spiel.python.games
from open_spiel.python.observation import make_observation
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.pytorch.mutations import *
from open_spiel.python.examples.ppo_utils import EnvParams
from open_spiel.python.examples.ppo_eval import eval_agents_parallel
from open_spiel.python.env_decorator import *

from auctions.webutils import *
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
import numpy as np 
import pandas as pd
from absl import logging
import tempfile
import subprocess

import open_spiel.python.games
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.algorithms.exploitability import nash_conv, best_response

import bokeh
from bokeh.io import curdoc
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import row, column
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap, factor_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
from bokeh.models import LinearAxis, Range1d
from bokeh.models import ColumnDataSource, FactorRange

from bokeh.resources import CDN
from bokeh.embed import file_html

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from django.db.models import F
from auctions.models import *

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from open_spiel.python.examples.ubc_cma import *

import itertools
output_notebook()
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.env_and_policy import *
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent

Loading BokehJS ...

In [11]:
# GAME_NAME = 'confused_national'
# GAME_NAME = 'all_confused_with_local'
# GAME_NAME = '2rvn'
# GAME_NAME = 'all_confused'
# GAME_NAME = 'regional_vs_national'
# GAME_NAME = 'very_small_sats_complete_symmetric'
# GAME_NAME = 'may17'


# GAME_NAME = 'may17'



# PYSATS = '/apps/sats/python'
# !python {PYSATS}/pysats.py --config_file {PYSATS}/{GAME_NAME}.yml --seed 777 --output_file {PYSATS}/{GAME_NAME}.json
# GAME_NAME = f'{PYSATS}/{GAME_NAME}'

In [12]:
# # GAME_NAME = 'sats_2regions_2licenses_2players_5types'
# GAME_NAME = 'confused_national'
# # GAME_NAME = '/apps/sats/python/regional_vs_national'

In [54]:
GAME_NAME = 'jun2/jun2_4_base'
# GAME_NAME = 'jun5/jun5_1_base'


In [55]:
game = pyspiel.load_game('python_clock_auction', dict(filename=f'{GAME_NAME}.json'))

In [56]:
FACTOR_OPENING_PRICES = True
# FACTOR_OPENING_PRICES = False

In [57]:
# Stuff that doesn't chagne per player
packages = game.auction_params.all_bids
pure_packages_indices = np.where(np.count_nonzero(packages, axis=1) == 1)
pure_packages = packages[pure_packages_indices]
products = [num_to_letter(i) for i in range(game.auction_params.num_products)]
quantities = list(map(str, range(1, max(game.auction_params.licenses) + 1)))
max_quantity = max(game.auction_params.licenses)
colors = Category10_10[:len(quantities)]
player_types = game.auction_params.player_types

# TODO: Should assert this only works for Enumerated Bidder
for player_index, player_type in player_types.items():
    num_types = len(player_types[player_index])

    records = []
    
    num_types = len(player_types[player_index])

    for type_index in range(num_types):
        bidder = player_types[player_index][type_index]['bidder']
        values = bidder.get_profits(game.auction_params.opening_prices) if FACTOR_OPENING_PRICES else bidder.get_values()
        values = values[pure_packages_indices]
        
        for package_index, pure_package in enumerate(pure_packages):
            product_index = np.nonzero(pure_package)[0][0]
            q = pure_package[product_index]
            record = dict(x=(game.auction_params.license_names[product_index], f'{type_index}_{bidder.name}' or str(type_index)), q=q, value=values[package_index])
            records.append(record)
    
    df = pd.DataFrame.from_records(records).pivot(index='x', columns='q')
    df.columns = map(str,df.columns.get_level_values(1))
    df = df.reset_index()
    marginal_df = df.copy()
    marginal_df["1"] = np.maximum(marginal_df["1"], 0)
    for z in range(2, max_quantity + 1):
        marginal_df[str(z)] = np.maximum(0, df[str(z)] - df[str(z - 1)])

    df = marginal_df
    
    if not FACTOR_OPENING_PRICES and (df.select_dtypes(include=np.number) < 0).sum().sum():
        # for pkg, val in zip(game.auction_params.all_bids,bidder.values):
        #     print(pkg, val)
        raise ValueError("No negative numbers expected!")
    
    source = ColumnDataSource(df)
    factors = df['x'].values

    p = figure(x_range=FactorRange(*factors), height=250, toolbar_location=None, tools="hover", tooltips="$name: @$name")

    p.vbar_stack(quantities, x='x', width=0.9, alpha=0.5, color=colors, source=source, legend_label=quantities)

    # p.y_range.start = 0
    # p.y_range.end = 18
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_center"
    p.legend.click_policy = 'mute'
    p.legend.orientation = "horizontal"
    p.add_layout(p.legend[0], 'right')
    p.title.text = f'Player {player_index}'

    # TODO: Add total value to hover, not just marginal
    show(p)


In [69]:
print(game.auction_params.opening_prices)
game.auction_params.licenses
print(game.new_initial_state().child(0).child(1).bidders[1].bidder.get_values())
game.new_initial_state().child(0).child(1).bidders[1].bidder.all_bids

[32, 19]
[ 0 17 24 33 55 61 47 69 76]


array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

In [60]:
df, combo_to_score, scorer = efficient_allocation(game, factor_in_opening_prices=FACTOR_OPENING_PRICES)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 174.21it/s]


In [ ]:
pd.DataFrame(*df['allocation'].values)

In [ ]:
worst_case_profit, allocation = efficient_allocation_avg_type(game, factor_in_opening_prices=FACTOR_OPENING_PRICES)

In [ ]:
allocation

In [ ]:
worst_case_profit / df['score'].mean()

In [ ]:
worst_case_profit

In [ ]:
df['score'].mean()

In [ ]:
df

In [ ]:
# TODO: Only works for 2p
df[['p0_type', 'p1_type']] = pd.DataFrame(df['combo'].apply(lambda x: x[:2]).tolist())
df[['p0_allocation', 'p1_allocation']] = pd.DataFrame(df['allocation'].apply(lambda x: x[:2]).tolist())
df[['r', 'g', 'b']] = pd.DataFrame(df['p0_allocation'].tolist()) / 3

plt_rgb = df[['r', 'g', 'b']].values
plt_x = df['p0_type'].values
plt_y = df['p1_type'].values
plt_colors = np.zeros((max(plt_x)+1, max(plt_y)+1, 3))
plt_colors[plt_y, plt_x, :] = plt_rgb
plt.imshow(plt_colors, interpolation='nearest')
plt.xlabel('Player 0 type')
plt.ylabel('Player 1 type')
plt.show()

In [ ]:
plt_colors[1, 2]

In [ ]:
plt_colors[0, 2]

In [ ]:
df

In [ ]:
df['allocation'].value_counts()

In [ ]:
a = game.auction_params.player_types[0][0]['bidder'].value_for_package([1,2,1])
b = game.auction_params.player_types[1][0]['bidder'].value_for_package([2,1,2])
print(a, b, a+b)

In [ ]:
a = game.auction_params.player_types[0][0]['bidder'].value_for_package([0,2,0])
b = game.auction_params.player_types[1][0]['bidder'].value_for_package([3,1,3])
print(a, b, a+b)

In [ ]:
game.auction_params.player_types[1][0]['bidder'].value_for_package([3,0,0])

In [ ]:
df

In [ ]:
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.env_and_policy import *
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent

In [ ]:
game.auction_params.increment = 0.05

In [ ]:
all_allocations = []
for combo in tqdm(type_combos(game)):
    history_prefix = [combo[p]['index'] for p in range(game.num_players())]
    named_combo = [combo[p]['bidder'].name for p in range(game.num_players())]
    env = EnvParams(track_stats=True, history_prefix=history_prefix, num_envs=1, normalize_rewards=False).make_env(game)
    env_and_policy = EnvAndPolicy(env=env, agents=[StraightforwardAgent(0, game), StraightforwardAgent(1, game)], game=game)
    evaluation = run_eval(env_and_policy, num_samples=10)
    allocations = []
    for player in range(game.num_players()):
        allocation = np.array(evaluation['allocations'][player]).mean(axis=0)
        allocations.append(allocation)
    all_allocations.append((named_combo, np.round(allocations, 2)))

In [ ]:
all_allocations

In [ ]:
# Iterate over cartesian product of types


In [ ]:
history_prefix = [1,1]
named_combo = ['n', 'n']
env = EnvParams(track_stats=True, history_prefix=history_prefix, num_envs=1, normalize_rewards=False).make_env(game)
env_and_policy = EnvAndPolicy(env=env, agents=[StraightforwardAgent(0, game), StraightforwardAgent(1, game)], game=game)
evaluation = run_eval(env_and_policy, num_samples=2)


In [ ]:
x = [(1,2,3), (4,5,6)]
for item in x:
    x.remove((4,5,6))

In [ ]:
x

In [16]:
s= game.new_initial_state()

In [18]:
s.clone?

Docstring: clone(self: pyspiel.State) -> pyspiel.State
Type:      method
